In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV


from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, ShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import pandas as pd
# preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit

# models
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor 
from sklearn.neural_network import MLPRegressor


#pipelines
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer

import pickle5 as pickle
import itertools
from joblib import dump,load

In [11]:

def get_occupancy_df(line,cleaned_df = None):
    '''calculates the occupancy column for passed dataframe(9292 or arriva) and adds it to that dataframe '''

    #CHECK TO ENSURE INPUT CLEANED DATAFRAME IS PASSED AS PART OF THE ARGUMENTS
    if cleaned_df is None:
        return None
    
    #LIST OF VALID LINE NUMBERS
    # CURRENTLY ONLY WORKING FOR LINE 4 OF BREDA 
    line_names = ['4']

    # CHECK TO ENSURE VALID LINE NUMBER PASSED IN PARAMETERS
    if line not in line_names:
        print("Invalid Line Number")
        return None
   
    #ORDER OF STOPS BELOW NEED TO MATCH LINE AND MODALITY GIVEN IN THE PARAMETERS IN THIS CASE: 2
    line_stops = [['Breda, Dreef','Breda, Nieuwe Heilaarstraat','Breda, Woonboulevard','Breda, Liesboslaan','Breda, Ambachtenlaan',
    'Breda, Doelen','Breda, Hovenierstraat','Breda, Burgemeester Sutoriusstraat','Breda, Flierstraat','Breda, Mgr.Nolensplein',
    'Breda, Heuvelbrink','Breda, Dr. Struyckenplein','Breda, Bontekoestraat','Breda, Amphia Zkh. Langendijk',
    'Breda, Langendijk','Breda, Graaf Hendrik III Laan','Breda, Grote Spie','Breda, Irenestraat',
    'Breda, Markendaalseweg','Breda, Centrum','Breda, Vlaszak','Breda, Centraal Station','Breda, Belcrumweg',
    'Breda, Konijnenberg','Breda, Spinveld','Breda, Donk','Breda, Heienlangdonk','Breda, Somerweide','Breda, Noortberghmoeren',
    'Breda, Cannaertserf','Breda, Komoord','Breda, Dwarsdijk','Breda, Emerparklaan','Breda, Heksenwiellaan']]
    
     
        
    #CREATE DICTIONARY OF ORDERED STOPS SO THAT WE CAN MATCH GIVEN SOURCE AND DESTINATION
    line_stop_dict = {line_name:line_stop for (line_name,line_stop) in zip(line_names,line_stops)} 
    for key in line_stop_dict.keys():
        line_stop_dict[key] = {stop.lower():num for (num,stop) in enumerate(line_stop_dict[key])}

    #INITIALIZE DATAFRAME OBJECT TO BE RETURNED AS THE OCCUPANCY DATAFRAME
    occupancy_df = pd.DataFrame(columns = ['Line','Direction','Date','Station','Occupancy'])

    #ITERATE OVER THE LINES PARAMTER PASSED TO THIS FUNCTION
    #ALL POSSIBLE VALUES: '4'
    for line in lines:

            #LIST WHICH STORES UNIQUE SOURCE-DESTINATION PAIRS
            stops_direction_pair = list()
            
            # 1: GENERATE ALL POSSIBLE PERMUTATIONS OF LENGTH 2 FROM ALL THE STOPS ON THE CURRENT LINE:
            # 2: ITERATE OVER THIS COLLECTION, SEPARATING ALL PAIRS BASED ON DIRECTION (SOURC<DEST:ORIGINAL, SOURC>DEST:REVERSE)
            for elem in itertools.permutations(line_stop_dict[line].items(),2):
                if elem[0][1] < elem[1][1]:
                    stops_direction_pair.append((elem[0],elem[1],'original')) 
                if elem[0][1] > elem[1][1]:
                    stops_direction_pair.append((elem[0],elem[1],'reverse'))
            
            #ITERATE OVER ALL POSSIBLE DIRECTION VALUES: 'original','reverse'
            for direction in directions:
                
                # SELECT ALL SOURCE-DESTINATION PAIRS WHERE THE DIRECTION MATCHES THE CURRENT DIRECTION 
                stops= [elem for elem in stops_direction_pair if elem[2]==direction]
                
                #TEMPORARY DATAFRAME TO STORE THE DATAFRAME SLICE FOR CURRENT OCCUPANCY CALCULATION 
                temp_df = pd.DataFrame()
                
                #ITERATE OVER ALL SELECTED SOURCE-DEST PAIRS, APPENDING ALL ROWS FOUND FROM THE CLEANED INPUT DATAFRAME IN THIS MANNER 
                for elem in stops:
                    temp_df = temp_df.append(cleaned_df[(cleaned_df['Source']==elem[0][0]) &(cleaned_df['Destination']==elem[1][0])])
                
                # ADDING SOURCE NUMBER AND DESTINATION NUMBER COLUMNS DENOTING THE POSITION OF THE STOP IN THE TRIP
                temp_df['Source Num'] = [line_stop_dict[line][stop] for stop in temp_df['Source']] 
                temp_df['Destination Num'] = [line_stop_dict[line][stop] for stop in temp_df['Destination']]
                
                #ITERATE OVER ALL UNIQUE DATES IN THE SELECTED DATAFRAME
                for date in temp_df['Date'].unique():

                    # ITERATE OVER ALL UNIQUE LOCATIONS PRESENT IN THE SELECTED DATAFRAME
                    for stop in pd.Index(temp_df['Source']).union(pd.Index(temp_df['Destination'])).unique():

                        #CALCULATE OCCUPANCY FOR THE SELCTED STOP BASED ON THE DIRECTION OF THE CURRENT SELECTED PAIR
                        # CALCULATIONS OF OCCUPANCY ARE DIFFERENT BUT SYMMETRIC FOR BOTH DIRECTIONS
                        if direction == 'original':
                            occupancy = temp_df[(temp_df['Date'] == date)&( temp_df['Source Num']  <= line_stop_dict[line][stop] )]['Count'].sum() - temp_df[(temp_df['Date'] == date) & (temp_df['Destination Num'] < line_stop_dict[line][stop]) ]['Count'].sum()
                        if direction == 'reverse':
                            occupancy = temp_df[(temp_df['Date'] == date)&( temp_df['Source Num']  >= line_stop_dict[line][stop] )]['Count'].sum() - temp_df[(temp_df['Date'] == date) & (temp_df['Destination Num'] > line_stop_dict[line][stop]) ]['Count'].sum()

                        #APPENDING A ROW TO THE OCCUPANCY DATAFRAME BASED ON THE CURRENT LINE,DIRECTION,DATE,STATION AND STATION NUM
                        occupancy_df = occupancy_df.append({'Line':line,'Direction':direction,'Date':date,'Station':stop,'Station Num':int(line_stop_dict[line][stop]),'Occupancy':occupancy},ignore_index=True)

    #ADD DAY COLUMN TO OCCUPANCY DATAFRAME
    occupancy_df['Day']=pd.Series(pd.DatetimeIndex(occupancy_df['Date'])).dt.day_name()

    return occupancy_df


In [12]:
def get_merged_df(occupancy_9292,occupancy_arriva):
    '''returns merged dataset after matching 9292 and arriva dataset rows'''
    
    return pd.merge(occupancy_9292,occupancy_arriva,how='inner',on=['Line','Direction','Date','Day','Station Num','Station'])

In [13]:
def get_candidate_models(x,y,params=None):
    '''returns a dictionary of 3 fitted candidate models'''
    
    if params is None:
        return None

    regressors = {'Default MLP':MLPRegressor().fit(x.reshape(-1,1),y),
              'Tuned MLP': GridSearchCV(MLPRegressor(),params).fit(x.reshape(-1,1),y),
              'Linear Regressor': LinearRegression().fit(x.reshape(-1,1),y)
             }
    return regressors

In [14]:
def regression_scores(x,y,regressors):
    ''' returns a dataframe containing r2 validation scores for a list of regressors  '''

    results = dict() 
    
    for (model_name,model_pipe) in regressors.items():
        for metric in ['r2']: 
            results[model_name]=np.mean(cross_validate(model_pipe,x.reshape(-1,1), y, scoring=metric,cv=2, return_train_score=False)['test_score'])

    res = pd.DataFrame(data=results,index=pd.Series([0]))
    res['metrics'] = pd.Series(['r2'])

    return res

In [15]:
def regression_plot(x,y,xlabel='X',ylabel='Y',logy=False,regressors=None):
    '''creates a scatter plot of all passed samples along with predictions of of all passed regressors'''

    fig, ax = plt.subplots()    
    ax.plot(x,y, linewidth=0, marker='v', label='Data points')
       
    if regressors is not None:

        max_x = np.max(x)
        for regressor in regressors.items():
            #regressor = regressor[1].fit(x.reshape(-1,1),y)
            ax.plot(range(int(max_x)), regressor[1].predict(np.array(range(int(max_x))).reshape(-1,1)) ,label=regressor[0])

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if logy:
        ax.set_yscale('log')
    ax.legend(facecolor='white')

In [18]:
def get_occupancy_prediction(line, modality, day,hour,source,destination,merged_df,cleaned_9292,cleaned_arriva,model):
    if source == destination:
        print('Source and Destination cannot be the same')
        return -1 
    if line != '4':
        print('The MVP supports only line 4 currently')
        return -1
    
    #LINE TO SERVE RESULTS FOR, MUST BE MATCHING DATA IN OCCUPANCY DATAFRAMES GIVEN 
    line_names = ['4']

    #ORDER OF STOPS BELOW NEED TO MATCH LINE AND MODALITY GIVEN IN THE PARAMETERS IN THIS CASE: 2
    line_stops = [['Breda, Dreef','Breda, Nieuwe Heilaarstraat','Breda, Woonboulevard','Breda, Liesboslaan','Breda, Ambachtenlaan',
    'Breda, Doelen','Breda, Hovenierstraat','Breda, Burgemeester Sutoriusstraat','Breda, Flierstraat','Breda, Mgr.Nolensplein',
    'Breda, Heuvelbrink','Breda, Dr. Struyckenplein','Breda, Bontekoestraat','Breda, Amphia Zkh. Langendijk',
    'Breda, Langendijk','Breda, Graaf Hendrik III Laan','Breda, Grote Spie','Breda, Irenestraat',
    'Breda, Markendaalseweg','Breda, Centrum','Breda, Vlaszak','Breda, Centraal Station','Breda, Belcrumweg',
    'Breda, Konijnenberg','Breda, Spinveld','Breda, Donk','Breda, Heienlangdonk','Breda, Somerweide','Breda, Noortberghmoeren',
    'Breda, Cannaertserf','Breda, Komoord','Breda, Dwarsdijk','Breda, Emerparklaan','Breda, Heksenwiellaan']]
    
    #CREATE DICTIONARY OF ORDERED STOPS SO THAT WE CAN MATCH GIVEN SOURCE AND DESTINATION
    line_stop_dict = {line_name:line_stop for (line_name,line_stop) in zip(line_names,line_stops)} 
    for key in line_stop_dict.keys():
        line_stop_dict[key] = {stop.lower():num for (num,stop) in enumerate(line_stop_dict[key])}

    try:
        #GET NUMBER OF SOURCE AND DESTINATION STOPS GIVEN
        source_num = line_stop_dict[line][source]
        dest_num = line_stop_dict[line][destination]
    except:
        return {'message': 'error', 'error': 'Please supply valid source and destination stops. For a list of valid values check source code'}


    #DEPENDING ON THE DIRECTION OF THE REQUESTED TRIP WE SELECT THE CORRESPONDING DATA
    #FOR EXAMPLE FOR source = 2 and destionation = 6 we select stations 2-5 for that day
    if source_num < dest_num:
        res_df = merged_df[(merged_df['Station Num']>=source_num)&(merged_df['Station Num']<dest_num)&(merged_df['Day']==day)]
    if source_num > dest_num:
        res_df = merged_df[(merged_df['Station Num']<=source_num)&(merged_df['Station Num']>dest_num)&(merged_df['Day']==day)]
        

    #9292 request counts for selected stations
    requests_count = np.array([res_df['Occupancy_x'][index] for index in res_df.index]).reshape(-1,1)

    #number of requests happening for that line, day and hour on average
    number_of_requests_hourly = cleaned_9292[(cleaned_9292['Line']==int(line))&(cleaned_9292['Day']==day)&(cleaned_9292['Modaliteitsnummer']==modality)&(cleaned_9292['Hour']==hour)].groupby('Hour').sum()['Count'].mean()
    
    #number of requests happening for that line and day on average
    number_of_requests_daily = cleaned_9292[(cleaned_9292['Line']==int(line))&(cleaned_9292['Day']==day)&(cleaned_9292['Modaliteitsnummer']==modality)].groupby('Hour').sum()['Count'].mean()

    #final result is the prediction of the model multiplied by the percentage of requests that happened in that hour
    result = np.mean(model.predict(requests_count)) * number_of_requests_hourly/number_of_requests_daily
    
    #in case no requests were found in that hour the result of the calclulation will be NaN so we return 0
    if np.isnan(result):
        result = 0

    return result
    

In [19]:
# Load 9292 and Arriva Data:
cleaned_9292 = pd.read_pickle('final_9292')
cleaned_arriva = pd.read_pickle('final_arriva')

#RUN THE FOLLOWING LINES TO CREATE THE OCCUPANCY DATASETS IF NEEDED. THE PROCESS TAKES TIME
#Create Occupancy Data from 9292 and Arriva Data:
# occupancy_df_arriva = get_occupancy_df('4',cleaned_arriva[cleaned_arriva['Line']==4])
# occupancy_df_9292 = get_occupancy_df('4',cleaned_9292[(cleaned_9292['Line']==4)&(cleaned_9292['Modaliteitsnummer']==2)])

#RUN THE FOLLOWING LINES TO SAVE THE CREATED OCCUPANCY DATASETS IN A PICKLE FORMAT
#Store Occupancy Data as pickle:
# occupancy_df_arriva.to_pickle('occupancy_arriva')
# occupancy_df_9292.to_pickle('occupancy_9292')


#READ OCCUPANCY DATA FROM PICKLED FILE
occupancy_df_arriva = pd.read_pickle('occupancy_arriva')
occupancy_df_9292 = pd.read_pickle('occupancy_9292')

#RUN THE FOLLOWING LINES TO CREATE THE MERGED DATASET IF NEEDED.
# CREATE MERGED DATA FROM 9292 OCCUPANCY AND ARRIVA OCCUPANCY DATA:
#merged_df = get_merged_df(occupancy_df_9292,occupancy_df_arriva)

# RUN THE FOLLOWING LINES TO SAVE THE CREATED MERGED DATASET IN A PICKLE FORMAT
# STORE MERGED DATA AS PICKLE
#merged_df.to_pickle('merged_data')

# Read Merged Data from pickle:
merged_df = pd.read_pickle('merged_data')

# Create Parameter Space for Candidate MLP Models:
mlp_params = {  
            'hidden_layer_sizes':[(100,),(1000,)],
            'activation': ['identity', 'logistic', 'relu', 'tanh'],
            # 'mlp__solver':'adam',
            'alpha':[0.1,0.01,0.001],
            #'learning_rate':['constant'],
            'learning_rate_init':[0.1,0.01],
            # 'mlp__power_t':0.5,  assert(solver=='sgd' and learning_rate=='inv_scaling)        
            'max_iter':[2000]
#             'tol':[0.0001],
             # 'mlp__verbose':False,
            # 'mlp__warm_start':False,
            # 'mlp__momentum':0.9,
            # 'mlp__nesterovs_momentum':True,
            # 'mlp__early_stopping':False,
            # 'mlp__validation_fraction':0.05,
#             'epsilon':1e-08,
#             'beta_1': 0.9,
#             'beta_2':0.999,
#             'epsilon':1e-8       
        }
        
# Create Trained Candidate Models from Parameter Space:
#regressors = get_candidate_models(merged_df['Occupancy_x'].values,merged_df['Occupancy_y'].values,mlp_params)
# Store Candidate Models as pickle:
#dump(regressors,'candidate_models')
# Read Candidate Models from pickle :
regressors = load('candidate_models')

ValueError: unsupported pickle protocol: 5

In [9]:
!pip install pickle5

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
You should consider upgrading via the 'C:\ProgramData\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
regression_plot(merged_df['Occupancy_x'].values,merged_df['Occupancy_y'].values,'9292 Request Count','Arriva Bus Occupancy',regressors=regressors,logy=False)

NameError: name 'regression_plot' is not defined

In [ ]:
regression_scores(merged_df['Occupancy_x'].values,merged_df['Occupancy_y'].values,regressors)

In [16]:
get_occupancy_prediction(line='4',day='Monday',hour=0,source='breda, centrum',destination='breda, dreef',merged_df=merged_df,cleaned_9292=cleaned_9292,cleaned_arriva=cleaned_arriva,model=regressors['Tuned MLP'])

44.878249349195634